In [99]:
import psycopg2

### Sample postgres table:

```
CREATE TABLE edge_table (
    id BIGSERIAL,
    source BIGINT,
    target BIGINT,
    cost FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT
);
```

### Postgres connection

In [100]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow",
    user="postgres",
    password="postgres")

### Data loading

In [101]:
conn.autocommit = True
cursor = conn.cursor() 

In [106]:
drop_table = "DROP TABLE IF EXISTS edge_table CASCADE;"

create_table = """
    CREATE TABLE edge_table (
    id BIGSERIAL,
    osmid BIGINT,
    source BIGINT,
    target BIGINT,
    length FLOAT,
    name VARCHAR,
    highway VARCHAR,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT);
"""

create_extension = """
    CREATE EXTENSION IF NOT EXISTS pgrouting CASCADE;
    CREATE EXTENSION IF NOT EXISTS postgis;
"""

create_index = """
    CREATE INDEX source_idx ON edge_table (source);
    CREATE INDEX target_idx ON edge_table (target);
"""

### Drop table (if exists)

In [107]:
cursor.execute(drop_table)

### Create extensions (pgRouting and postgis)

In [108]:
cursor.execute(create_extension)

### Create table

In [109]:
cursor.execute(create_table)

### Load data from csv

In [110]:
# TODO: remember to change path to .csv file
csv_file_name = "/home/pcend/Piotr/kod/lisboa-routing/data/krakow/edge_table_sql.csv"
copy_csv_query = "COPY edge_table(source, target, osmid, length, name, highway, x1, y1, x2, y2) FROM STDIN DELIMITER ',' CSV HEADER"

In [111]:
cursor.copy_expert(copy_csv_query, open(csv_file_name, "r"))

### Create indexes on source and target

In [112]:
cursor.execute(create_index)

### Search queries

In [113]:
ids = (356926768, 2104495834) # short (zakamycze, betel)
# ids = (13822575, 11286442329) # random small

In [137]:
dijkstra = f"""
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

dijkstra_benchmark = f"""
    EXPLAIN ANALYZE 
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar = f"""
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar_benchmark = f"""
    EXPLAIN ANALYZE
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

In [138]:
print(astar)
print(dijkstra)


    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    356926768, 2104495834,
    directed => false);


    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    356926768, 2104495834,
    directed => false);



In [133]:
cursor.execute(astar)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

In [135]:
cursor.execute(dijkstra)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

### Disconnect

In [1]:
conn.commit() 
conn.close() 

NameError: name 'conn' is not defined